<a href="https://colab.research.google.com/github/david900125/113NCHU_Big-Data-Analytics-for-Finance/blob/main/week12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install google-api-core

In [18]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=1ec60c621b16c14844819a66111b11e70b17f01e8ea397062c4c48c16d4eff14
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [19]:
import yfinance as yf
import pandas as pd
import requests
import numpy as np
from ta.trend import SMAIndicator
from ta.momentum import RSIIndicator
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import google.generativeai as genai

genai.configure(api_key="")
model = genai.GenerativeModel("gemini-1.5-flash")

In [23]:
def fetch_three_years_data(stock_symbol):
    """
    獲取過去三年的股票數據。
    """
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=3*365)).strftime('%Y-%m-%d')

    stock = yf.Ticker(stock_symbol)
    data = stock.history(start=start_date, end=end_date)
    return data

def calculate_technical_indicators(data):
    """
    計算技術指標：簡單移動平均線和 RSI。
    """
    sma = SMAIndicator(data['Close'], window=50)
    data['SMA_50'] = sma.sma_indicator()

    rsi = RSIIndicator(data['Close'])
    data['RSI'] = rsi.rsi()

    return data


def summarize_stock_data(data):
    """
    將股價數據轉換為簡潔的文本描述，供語意分析使用。
    """
    recent_close = data['Close'].iloc[-1]
    sma_50 = data['SMA_50'].iloc[-1]
    rsi = data['RSI'].iloc[-1]

    trend = "upward" if data['Close'].iloc[-1] > data['Close'].iloc[-30] else "downward"

    summary = (
        f"The stock's recent closing price is {recent_close:.2f}. "
        f"The 50-day simple moving average is {sma_50:.2f}, and the RSI is {rsi:.2f}. "
        f"The stock has been showing a {trend} trend over the past month."
    )
    return summary

def analyze_sentiment_with_stock_data(api_key, news_titles, stock_summary):
    # 結合股價數據與新聞
    combined_text = stock_summary + "\n\n" + "Recent news:\n" + "\n".join(news_titles)

    response = model.generate_content(f"{combined_text} Provide financial sentiment analysis and investment recommendations based on the stock data and news provided.")
    return response.text

if __name__ == "__main__":
    # 配置
    stock_symbol = "AAPL"
    recent_news = [
        "Apple announces record earnings for the quarter.",
        "New iPhone sales exceed expectations.",
        "Apple faces supply chain challenges amid rising demand."
    ]

    # 獲取三年股價數據
    print("Fetching historical stock data...")
    stock_data = fetch_three_years_data(stock_symbol)
    stock_data = calculate_technical_indicators(stock_data)

    # 整理股價數據摘要
    stock_summary = summarize_stock_data(stock_data)
    print("Stock Summary:\n", stock_summary)

    # 分析情感與生成報告
    print("Analyzing sentiment and generating report...")
    report = analyze_sentiment_with_stock_data(gemini_api_key, recent_news, stock_summary)

    # 顯示報告
    print("Final Report:")
    print(report)


Fetching historical stock data...
Stock Summary:
 The stock's recent closing price is 242.65. The 50-day simple moving average is 229.27, and the RSI is 73.03. The stock has been showing a upward trend over the past month.
Analyzing sentiment and generating report...
Final Report:
## Financial Sentiment Analysis and Investment Recommendations for Apple Stock

**Data Summary:**

* **Closing Price:** $242.65 (relatively high)
* **50-Day SMA:** $229.27 (price is above the SMA, indicating bullish momentum)
* **RSI:** 73.03 (approaching overbought territory, suggesting potential near-term correction)
* **Trend:** Upward over the past month
* **News:** Strong earnings, exceeding iPhone sales expectations, but supply chain challenges.

**Sentiment Analysis:**

The overall sentiment is currently **positive but cautiously optimistic.**  The strong financial results (record earnings and exceeding iPhone sales expectations) are overwhelmingly bullish. This suggests strong underlying fundamentals 